In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

data_sets = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    
    for filename in filenames:
        print(filename)
        if filename == "test_essays.csv":
            test = pd.read_csv(os.path.join(dirname, filename))
        if filename == "train_essays.csv":
            train = pd.read_csv(os.path.join(dirname, filename))
        if filename == "train_essays_RDizzl3_seven_v2.csv":
            generated_text = pd.read_csv(os.path.join(dirname, filename))
        if filename == "train_v2_drcat_02.csv":
            generated_text2 = pd.read_csv(os.path.join(dirname, filename))
        data_sets.append(pd.read_csv(os.path.join(dirname, filename)))
    print(f"{len(data_sets)} data sets loaded in.")
    
    
#test = train[10:100]
#sample_sub, training_prompts, test, train, _, _, _, generated_text = data_sets

0 data sets loaded in.
sample_submission.csv
train_prompts.csv
test_essays.csv
train_essays.csv
4 data sets loaded in.
train_v2_drcat_02.csv
5 data sets loaded in.
train_essays_RDizzl3_seven_v2.csv
train_essays_7_prompts_v2.csv
train_essays_7_prompts.csv
train_essays_RDizzl3_seven_v1.csv
9 data sets loaded in.


In [2]:
print("Supplementary dataset features:")
print(generated_text.dtypes)
print("Supplementary dataset2 features:")
print(generated_text2.dtypes)
print("\nOriginal dataset features:")
print(train.dtypes)


Supplementary dataset features:
text     object
label     int64
dtype: object
Supplementary dataset2 features:
text             object
label             int64
prompt_name      object
source           object
RDizzl3_seven      bool
dtype: object

Original dataset features:
id           object
prompt_id     int64
text         object
generated     int64
dtype: object


As we can see, we need to make it so that these share a universal format so that we can combine the data together.

In [3]:
# Rename the label column to match original dataset
generated_text.rename(columns={'label': 'generated'}, inplace=True)
generated_text

,text,generated
0,Cars. Cars have been around since they became ...,0
1,Transportation is a large necessity in most co...,0
2,"""America's love affair with it's vehicles seem...",0
3,How often do you ride in a car? Do you drive a...,0
4,Cars are a wonderful thing. They are perhaps o...,0
...,...,...
17246,"Dear Senator,\n\nI am writing to you today to ...",1
17247,"Dear Senator,\n\nI am writing to you today to ...",1
17248,"Dear Senator,\n\nI am writing to you today to ...",1
17249,"Dear Senator,\n\nI am writing to you today to ...",1


In [4]:
# Drop the two columns we don't need for training
generated_text2.drop('source', axis='columns', inplace=True)
generated_text2.drop('prompt_name', axis='columns', inplace=True)
generated_text2.drop('RDizzl3_seven', axis='columns', inplace=True)
generated_text2.rename(columns={'label': 'generated'}, inplace=True)
generated_text2

,text,generated
0,Phones\n\nModern humans today are always on th...,0
1,This essay will explain if drivers should or s...,0
2,Driving while the use of cellular devices\n\nT...,0
3,Phones & Driving\n\nDrivers should not be able...,0
4,Cell Phone Operation While Driving\n\nThe abil...,0
...,...,...
44863,"Dear Senator,\n\nI am writing to you today to ...",1
44864,"Dear Senator,\n\nI am writing to you today to ...",1
44865,"Dear Senator,\n\nI am writing to you today to ...",1
44866,"Dear Senator,\n\nI am writing to you today to ...",1


In [5]:
train.drop('id', axis='columns', inplace=True)
train.drop('prompt_id', axis='columns', inplace=True)
train

,text,generated
0,Cars. Cars have been around since they became ...,0
1,Transportation is a large necessity in most co...,0
2,"""America's love affair with it's vehicles seem...",0
3,How often do you ride in a car? Do you drive a...,0
4,Cars are a wonderful thing. They are perhaps o...,0
...,...,...
1373,There has been a fuss about the Elector Colleg...,0
1374,Limiting car usage has many advantages. Such a...,0
1375,There's a new trend that has been developing f...,0
1376,As we all know cars are a big part of our soci...,0


In [6]:
training_set = pd.concat([generated_text, train])
training_set = pd.concat([generated_text2, training_set])

In [7]:
training_set['generated'].value_counts()

generated
0    42993
1    20504
Name: count, dtype: int64

## Undersample to get an even distribution

In [8]:
pos = training_set[training_set['generated']==1]
neg = training_set[training_set['generated']==0]
neg = neg.sample(n=len(pos), random_state = 21)

training_set = pd.concat([pos, neg])

In [9]:
training_set['generated'].value_counts()

generated
1    20504
0    20504
Name: count, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
import keras_nlp
import keras_core as keras

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
   

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import EarlyStopping

Using TensorFlow backend


In [11]:
isRealTest = True

if isRealTest:
    X_train = training_set['text'].values
    y_train = training_set['generated'].values
    
    X_test = test['text'].values
    X_testIDs = test['id']
    
    
else:
    X = training_set['text']
    y = training_set['generated']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [12]:
max_words = 10000
padding_length = 200

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Tokenize them into vectors
# Each unique word represented by an integer
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad our data to ensure consistent length
X_train_padded = pad_sequences(X_train_sequences, maxlen=padding_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=padding_length, padding='post', truncating='post')

In [13]:
print(X_test)

['Aaa bbb ccc.' 'Bbb ccc ddd.' 'CCC ddd eee.']


### Visualizing our preprocessing

Now we can see the three steps our data goes through


In [14]:
# essay_number = 5

# print("Step 1: Raw Text\n"+ str(X_train[5]))
# print("\n\nStep 2: Tokens\n" + str(X_train_sequences[essay_number]))
# print("\n\nStep 3: Padded Set\n" + str(X_train_padded[essay_number]))

# print("Generated? " + str(bool(y_train[essay_number])))

### Feature Engineering

At this point, we can being to engineer out some new features that might give our model a nuanced idea of what is happening in the data.

Some ideas I have right now:
* Syntax and Grammar:
Use language processing tools to analyze the syntactic structure and grammatical correctness of sentences. AI-generated text might exhibit patterns that deviate from human-like syntax.

* Vocabulary Complexity:
Measure the complexity of vocabulary used in the text. AI models might generate text with either overly complex or simplistic language compared to typical human writing.
* Semantic Coherence:
Assess the coherence of sentences in terms of semantic meaning. Human-generated text tends to have a more natural flow of ideas and coherent structure.
* Common Phrases and Idioms:
Check for the use of common phrases, idioms, and cultural references that are typically found in human communication. AI models may struggle to mimic these accurately.
* Context Understanding:
Analyze the text for evidence of a deep understanding of context. Human writers often incorporate a broader contextual understanding into their writing that AI models may find challenging to replicate.
* Creativity and Originality:
Assess the level of creativity and originality in the text. Human writers often bring a unique perspective and creativity to their writing that may be distinct from AI-generated content.
* Length and Structure:
Examine the length and structure of sentences and paragraphs. AI models might produce text with consistent or repetitive structures that differ from the varied structures often found in human writing.
* Consistency in Style:
Evaluate the consistency in writing style throughout the text. Human writers may exhibit variations in style, tone, and expression, while AI-generated text might be more consistent.
* Awareness of Current Events:
Check for references to recent events or time-sensitive information. AI models might not have up-to-date information or an understanding of current events.
* Metaphors and Figurative Language:
Analyze the use of metaphors, similes, and other forms of figurative language. Human writers often use these elements to convey meaning in a nuanced way.

To combine these together, we can have a layer that combines our smaller parallel nodes and gives them different weight. I still believe that our raw text will have a large impact on its performance, but we want to supplement it with more valuable datapoints. 

From there, we need to bucket certain features together or keep them all seperate.

In [15]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=200),
    LSTM(64),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=1e-4), loss =tf.keras.losses.BinaryCrossentropy(), metrics=[BinaryAccuracy()])

In [ ]:
y_train = y_train.astype(np.float32)
if not isRealTest:
    y_test  = y_test .astype(np.float32)

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(X_train_padded, y_train, epochs=5, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/5
1154/1154 [==============================] - 126s 106ms/step - loss: 0.2260 - binary_accuracy: 0.9042 - val_loss: 0.0317 - val_binary_accuracy: 0.9937
Epoch 2/5
1154/1154 [==============================] - 121s 105ms/step - loss: 0.0420 - binary_accuracy: 0.9901 - val_loss: 0.0362 - val_binary_accuracy: 0.9932
Epoch 3/5
1097/1154 [===========================>..] - ETA: 5s - loss: 0.0261 - binary_accuracy: 0.9945

In [ ]:
if not isRealTest:
    # Evaluate the model on the test set
    loss, accurtestcy = model.evaluate(X_test_padded, y_test)
    print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Predictions on the test set
y_pred = model.predict(X_test_padded)

# Convert probabilities to class labels
y_pred_labels = np.round(y_pred)

In [ ]:
if not isRealTest:
    # Confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred_labels)
    print("Confusion Matrix:")
    print(conf_matrix)

    # Classification report
    class_report = classification_report(y_test, y_pred_labels)
    print("\nClassification Report:")
    print(class_report)

if isRealTest:
    submission = pd.DataFrame({'id': list(X_testIDs), 'generated': y_pred.flatten()})

In [ ]:
submission.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
submission